In [1]:
import tensor_comprehensions as tc
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
import matplotlib.pyplot as plt

import os
import sys
import logging
import time

In [2]:
# as recommended by the authors for better performance
tune_settings = {
    "threads": 32,
    "generations": 5,
    "pop_size": 15,
    "number_elites": 2
}

In [3]:
gconv_lang = tc.database['group_convolution']['lang']

In [4]:
# try the sizes given in the paper
# N = 32, G = 32, F = 16, C = 16, W = 14, H = 14

I = torch.randn(32, 32, 16, 14, 14).cuda()
W1 = torch.randn(32, 16, 16, 3, 3).cuda()
B = torch.randn(32, 16).cuda()

In [5]:
gconv = tc.define(gconv_lang, name='group_convolution')

In [5]:
gconv.autotune(I, W1, B,
               cache='cache/gconv1.tc',
               **tune_settings,
               options=tc.Options('conv'))

[INFO]: Autotuning cache will be saved to: cache/tall_conv.tc.cuda/options


In [6]:
# run twice to prep cuda
out = gconv(I, W1, B, cache='cache/gconv1.tc')
torch.cuda.synchronize()
out = gconv(I, W1, B, cache='cache/gconv1.tc')
torch.cuda.synchronize()
print(out.size())


torch.Size([16, 6, 86, 222])


In [7]:
# test performance using TC
torch.cuda.synchronize()

timings = np.zeros(50)

for i in range(50):
    start = time.perf_counter()
    output = gconv(I, W1, B, cache='cache/gconv1.tc')
    torch.cuda.synchronize()
    timings[i] = time.perf_counter() - start

total_elapsed = np.sum(timings)
print('total time: ' + str(total_elapsed))

# get percentile statistics
percentiles = np.percentile(timings, [0, 50, 90])
print(percentiles)


total time: 8.214661343
[ 0.01603009  0.01610681  0.01728153]


In [8]:
# test the torch equivalent

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.layer = nn.Sequential(
            nn.Conv2d(16, 16, 3, groups=32)
    
    def forward(self, x):
        return self.layer(x)

CN = ConvNet()
CN.cuda()

var_input = Variable(torch.randn(32, 512, 14, 14).cuda())
# run a couple times to work out start hiccups
CN(var_input)
torch.cuda.synchronize()
CN(var_input)
torch.cuda.synchronize()


# test performance
timings = np.zeros(50)

for i in range(50):
    start = time.perf_counter()
    output = CN(var_input)
    torch.cuda.synchronize()
    timings[i] = time.perf_counter() - start

total_elapsed = np.sum(timings)
print('total time: ' + str(total_elapsed))

# get percentile statistics
percentiles = np.percentile(timings, [0, 50, 90])
print(percentiles)

total time: 1.312579019
[ 0.00256825  0.00258939  0.00277763]
